In [23]:
import openai
import project_secrets
import sys
import os
from typing import List
from pprint import pprint
import ast
import json
# print(os.path.dirname(os.getcwd()))
# sys.path.append(os.path.dirname(os.getcwd()))

import time


In [24]:
openai.api_key = project_secrets.OPENAI_KEY
# print(project_secrets.OPENAI_KEY)

PATH_TO_DATA = os.path.join(os.getcwd(), "data")
# context_file = "context1.txt"
PATH_TO_CONTEXT_DIR = os.path.join(PATH_TO_DATA, "contexts")
print(PATH_TO_CONTEXT_DIR)

# data DIY
PATH_TO_DIR_COURSES = os.path.join(PATH_TO_DATA, "DIY")
print(PATH_TO_DIR_COURSES)
PATH_TO_DIR_COURSES_TXT = os.path.join(PATH_TO_DIR_COURSES, "courses_txt")
PATH_TO_DIR_COURSES_TAGS = os.path.join(PATH_TO_DIR_COURSES, "courses_tags")

CHATGPT_MODEL = "gpt-3.5-turbo"



/home/nalex/Desktop/different/feedback-system/ranking/generate_tags/data/contexts
/home/nalex/Desktop/different/feedback-system/ranking/generate_tags/data/DIY


In [25]:
def get_context(file_name: str):
    path_to_file = os.path.join(PATH_TO_CONTEXT_DIR, file_name)
    file = open(path_to_file, "r", encoding="utf-8")
    return file.read()

def get_txt_from_file(path : str):
#     print(path)
    with open(path, "r", encoding="utf-8") as file:
        text = file.read()
        return text
    
    return None

def get_all_courses(field_of_knowledge_list : List[str] = None):
    courses : Dict[str, str] = dict()
    
    if field_of_knowledge_list is None:
        field_of_knowledge_list = os.listdir(PATH_TO_DIR_COURSES_TXT)
        
        
    for field_of_knowledge in field_of_knowledge_list:
        path_to_sub_dir = os.path.join(PATH_TO_DIR_COURSES_TXT, field_of_knowledge)
        for sub_dir in os.listdir(path_to_sub_dir):
            path_to_dir_file = os.path.join(path_to_sub_dir, sub_dir)
            for file_name in os.listdir(path_to_dir_file):
                short_name = file_name.split('.')[0]

                path_to_file = os.path.join(path_to_dir_file, file_name)
                text = get_txt_from_file(path_to_file)
                
                courses[short_name] = text
                
    return courses

In [26]:
# courses = get_all_courses(["Алгебра"])
# pprint(courses)

In [27]:
def get_text_tags_from_course(course_text : str, context : str):

    messages = [ {"role" : "assistant", "content": context },
             {"role": "user", "content": course_text} ]
    
    completion = openai.ChatCompletion.create(
        model=CHATGPT_MODEL,
        messages=messages
    )

    return completion.choices[0].message.content

def change_inner_bracket(text : str):
    first_br_ind = None
    for i, ch in enumerate(text):
        if ch == '"':
            first_br_ind = i
    if first_br_ind is None:
        # WTF ????
        pass
    
    last_br_ind = None
    for i, ch in enumerate(reversed(text)):
        if ch == '"':
            last_br_ind = len(text) - i - 1
    
    for i in range(first_br_ind + 1, last_br_ind):
        if text[i] == '"':
            text[i] = "'"
    
    return text

def preprocessing(text : str):
#     text = text.replace("'", '"')
#     text = change_inner_bracket(text)
    
    return text

def print_pretty_array(arr : list):
    print('[')
    for s in arr:
        print(f'\t"{s}"')
    print(']')

In [28]:
# со статическими переменными функции какие-то пробелмы, сделаю через глобальные
TIME_BEFORE_SLEEP = None
def run_ChatGPT_for_text(short_course_name, course_text,
           file_context : str = None, is_print: bool = False):

    time_sleep = 21
    print(f"run_ChatGPT_for_text: заснул на {time_sleep} секунд, а то нас забанят :( !")
    time.sleep(time_sleep)
    print(f"run_ChatGPT_for_text: проснулся и готов делать запрос!")
    # __DELAY REQUEST FOR ChatGPT API
    
    
    context = get_context(file_context)
    print(f"ChatGPT request is started! {short_course_name}: {file_context}")
    text_tags = get_text_tags_from_course(course_text=course_text, context=context)
    text_tags = preprocessing(text_tags)
#     print(f"run_ChatGPT_for_text:{text_tags}")
    
    print("ChatGPT request is finished!")
    
    tags_str_list = ast.literal_eval(text_tags)
    if is_print:
        print_pretty_array(tags_str_list)
    
    return tags_str_list

In [29]:
def procceing_chatGPT_for_all_courses(field_of_knowledge_list : List[str] = None,
                                     rewrite : bool = False):
#     courses : Dict[str, str] = dict()
    
    if field_of_knowledge_list is None:
        field_of_knowledge_list = os.listdir(PATH_TO_DIR_COURSES_TXT)
        
        
    for field_of_knowledge in field_of_knowledge_list:
        path_to_sub_dir = os.path.join(PATH_TO_DIR_COURSES_TXT, field_of_knowledge)
        for sub_dir in sorted(os.listdir(path_to_sub_dir)):
            path_to_dir_file = os.path.join(path_to_sub_dir, sub_dir)
            for file_name in os.listdir(path_to_dir_file):
                short_name = file_name.split('.')[0]
                
                path_to_file = os.path.join(path_to_dir_file, file_name)
                # for tags
                path_to_tags_dir = os.path.join(PATH_TO_DIR_COURSES_TAGS, field_of_knowledge)
                if not os.path.exists(path_to_tags_dir):
                    os.mkdir(path_to_tags_dir)
                path_to_tags_subdir = os.path.join(path_to_tags_dir, sub_dir)
                if not os.path.exists(path_to_tags_subdir):
                    os.mkdir(path_to_tags_subdir)
                
                path_to_tags = os.path.join(path_to_tags_subdir, f"{short_name}.json")
                if os.path.exists(path_to_tags) and not rewrite:
                    continue                 
                    
                # __for tags
                
                
                cur_courses_text = get_txt_from_file(path_to_file)
                
                print(f"{short_name}: {path_to_tags}")

                tags_str_list = run_ChatGPT_for_text(short_name,
                                                     course_text=cur_courses_text, 
                                                     file_context="context2.txt")
                print()
                
                json.dump(tags_str_list, open(path_to_tags, "w", encoding="utf-8"),
                          indent=4, ensure_ascii=False)
    print(f"procceing_chatGPT_for_all_courses: FINISHED !!!")
                
    return 

In [30]:
# procceing_chatGPT_for_all_courses(field_of_knowledge_list=["Алгебра"])
procceing_chatGPT_for_all_courses()

HomolAlg: /home/nalex/Desktop/different/feedback-system/ranking/generate_tags/data/DIY/courses_tags/Алгебра/1/HomolAlg.json
run_ChatGPT_for_text: заснул на 21 секунд, а то нас забанят :( !
run_ChatGPT_for_text: проснулся и готов делать запрос!
ChatGPT request is started! HomolAlg: context2.txt
ChatGPT request is finished!

CommAlg: /home/nalex/Desktop/different/feedback-system/ranking/generate_tags/data/DIY/courses_tags/Алгебра/1/CommAlg.json
run_ChatGPT_for_text: заснул на 21 секунд, а то нас забанят :( !
run_ChatGPT_for_text: проснулся и готов делать запрос!
ChatGPT request is started! CommAlg: context2.txt
ChatGPT request is finished!

GaloisHom: /home/nalex/Desktop/different/feedback-system/ranking/generate_tags/data/DIY/courses_tags/Алгебра/1/GaloisHom.json
run_ChatGPT_for_text: заснул на 21 секунд, а то нас забанят :( !
run_ChatGPT_for_text: проснулся и готов делать запрос!
ChatGPT request is started! GaloisHom: context2.txt
ChatGPT request is finished!

LinAlg: /home/nalex/Deskt